<a href="https://colab.research.google.com/github/joshIsac/LargeLanguageModel/blob/main/llm__lab6_seqtoseqtransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
#importing library for Seq2seq transformer
import tensorflow as tf
from tensorflow import keras
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import accuracy_score

In [ ]:
#import the datasets for language translation using datasets library
import datasets
from datasets import load_dataset


data = load_dataset("Helsinki-NLP/opus_books", "ca-en")


In [ ]:
#tokenize the data
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ca-en")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
#tokenize the data
def tokenize_data(data):
  input_ids =[]
  attention_masks =[]
  for item in data:
    src_txt = item['translation']['ca']
    tgt_txt = item['translation']['en']
    encoded_dict = tokenizer(src_txt,return_tensors='pt',padding='max_length',max_length=128,truncation=True)
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)
input_ids,attention_masks = tokenize_data(data['train'])

In [ ]:
input_ids

array([[[ 4571,   164,  1633, ..., 55254, 55254, 55254]],

       [[  317,    77,   837, ..., 55254, 55254, 55254]],

       [[ 3653,   301,  2901, ..., 55254, 55254, 55254]],

       ...,

       [[  394,  9596,  7353, ..., 55254, 55254, 55254]],

       [[   86,  1569,   241, ..., 55254, 55254, 55254]],

       [[11758,    85,  4210, ..., 55254, 55254, 55254]]])

In [ ]:
attention_masks

array([[[1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0]],

       ...,

       [[1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0]],

       [[1, 1, 1, ..., 0, 0, 0]]])

In [81]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
model=TFAutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ca-en")
# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-ca-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [89]:
#prepare the dataset
train_dataset=tf.data.Dataset.from_tensor_slices((input_ids,attention_masks)).batch(32)

In [84]:
# Define loss and compile the model

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [92]:
model.fit(train_dataset,epochs=10)